**Analysis owner**: Felipe Matheus <br>
**Date**: 18/02/2021 <br>
**Related JIRA issue**:  https://hurbcom.atlassian.net/browse/ML-42

This notebook objective is to model the chats distribution logic that will be used as a feature to the attendants prediction model.

Once we have the amount of chats per week and our ML model predicting for each hour, we need to find the best way to distribute those chats for each one of the 168 hours of the week.

We are going to analyze the daily/"hourly" statistic distribution of chats during some date range.

# Setup

In [2]:
import pandas as pd
import holidays
import os
import json
from datetime import datetime

In [3]:
from src.DatasetLoader import DatasetLoader
from src.feature_engineering.FeatureEngineering import FeatureEngineering
from src.pre_processing.Processing import Processing
from src.metrics.Evaluations import Evaluations

%load_ext autoreload
%autoreload 2

In [4]:
with open('../configurations/variables.json') as json_file:
    var = json.load(json_file)
BUCKET = var.get('BUCKET')
FOLDER = var.get('FOLDER')
MILESTONES = var.get('MILESTONES')
CONVERT_WEEKDAY = var.get("CONVERT_WEEKDAY")

In [5]:
load = DatasetLoader()
feng = FeatureEngineering()
processing = Processing()
evaluations = Evaluations()

# Loading DataFrames
And adding weekdays and hours as columns
- All data until July 14
- All data until July 14 excluding holidays

In [6]:
df = feng.get_feature_engineered_df(
    processing.select_time_interval(
        load.load_query('kpis_chats_hour', BUCKET=BUCKET, FOLDER=FOLDER),
        MILESTONES['Q1_2020'][0],
        MILESTONES['Q1_2020'][1],
        'exclude')[['time','missed','week_day','chats', 'TMA', 'TME']],
    processing.select_time_interval(
        load.load_query('attendants_chats_hour', BUCKET=BUCKET, FOLDER=FOLDER),
        MILESTONES['Q1_2020'][0],
        MILESTONES['Q1_2020'][1],
        'exclude'
    )
)

File hurb_internal_dsc_files.notebooks_data/attendants-allocation/.kpis_chats_hour downloaded. Returning DataFrame and saving local pickle.
File hurb_internal_dsc_files.notebooks_data/attendants-allocation/.attendants_chats_hour downloaded. Returning DataFrame and saving local pickle.


In [7]:
df["week_day"] = df.index.weekday.astype(str).map(CONVERT_WEEKDAY)
df["hour"] = df.index.hour

In [8]:
df_no_hdays = processing.drop_days(df, [hd for hd in holidays.Brazil()[MILESTONES['CHAT_TEST_END']:'2020-12-31']])

## DataFrames visualization

In [9]:
df

,night,weekend,chats,TMA,TME,dropout,ct_agents,week_day,hour
time,,,,,,,,,
2019-06-16 00:00:00,True,True,24,11.275,19.697917,0.50,2,Sunday,0
2019-06-16 01:00:00,True,True,9,12.614815,9.209259,0.00,2,Sunday,1
2019-06-16 02:00:00,True,True,6,19.388889,0.761111,0.00,2,Sunday,2
2019-06-16 03:00:00,True,True,4,11.327778,0.695833,0.25,2,Sunday,3
2019-06-16 04:00:00,True,True,1,18.6,0.666667,0.00,1,Sunday,4
...,...,...,...,...,...,...,...,...,...
2021-07-14 01:00:00,True,False,26,18.634615,0.305128,0.00,21,Wednesday,1
2021-07-14 02:00:00,True,False,14,11.615476,0.313095,0.00,13,Wednesday,2
2021-07-14 03:00:00,True,False,12,11.140278,0.365278,0.00,10,Wednesday,3


In [10]:
df_no_hdays

,night,weekend,chats,TMA,TME,dropout,ct_agents,week_day,hour
time,,,,,,,,,
2019-06-16 00:00:00,True,True,24,11.275,19.697917,0.50,2,Sunday,0
2019-06-16 01:00:00,True,True,9,12.614815,9.209259,0.00,2,Sunday,1
2019-06-16 02:00:00,True,True,6,19.388889,0.761111,0.00,2,Sunday,2
2019-06-16 03:00:00,True,True,4,11.327778,0.695833,0.25,2,Sunday,3
2019-06-16 04:00:00,True,True,1,18.6,0.666667,0.00,1,Sunday,4
...,...,...,...,...,...,...,...,...,...
2021-07-14 01:00:00,True,False,26,18.634615,0.305128,0.00,21,Wednesday,1
2021-07-14 02:00:00,True,False,14,11.615476,0.313095,0.00,13,Wednesday,2
2021-07-14 03:00:00,True,False,12,11.140278,0.365278,0.00,10,Wednesday,3


# Getting percentual chats distributions
4 periods:
- All time
- Only before COVID - per1
- After COVID and before September - per2
- After September - per3
- After tests (March 2021) untill validation weekends '2021-06-16' - per4

With and without holidays

In [11]:
percentages = {
    "percentage_all": feng.get_specific_day_hour_percentages(df, [MILESTONES['CHAT_TEST_END'], '2021-06-16']),
    "percentage_all_no_hdays": feng.get_specific_day_hour_percentages(df_no_hdays, [MILESTONES['CHAT_TEST_END'], '2021-06-16']),
    "per_1": feng.get_specific_day_hour_percentages(df, [MILESTONES['CHAT_TEST_END'], MILESTONES['COVID']]),
    "per_1_no_hdays": feng.get_specific_day_hour_percentages(df_no_hdays,  [MILESTONES['CHAT_TEST_END'], MILESTONES['COVID']]),
    "per_2": feng.get_specific_day_hour_percentages(df, [MILESTONES['COVID'], MILESTONES["INSIDE_SALES_OFF"]]),
    "per_2_no_hdays": feng.get_specific_day_hour_percentages(df_no_hdays,  [MILESTONES['COVID'], MILESTONES["INSIDE_SALES_OFF"]]),
    "per_3": feng.get_specific_day_hour_percentages(df, [MILESTONES["INSIDE_SALES_OFF"], '2021-06-16']),
    "per_3_no_hdays": feng.get_specific_day_hour_percentages(df_no_hdays,  [MILESTONES["INSIDE_SALES_OFF"], '2021-06-16']),
    "per_4": feng.get_specific_day_hour_percentages(df, ['2021-03-01', '2021-06-16']),
    "per_4_no_hday": feng.get_specific_day_hour_percentages(df_no_hdays,  ['2021-03-01', '2021-06-16'])
}

## Percentage dict visualization
The percentage of each hour of each week day will be structured as nested dictionaries, as the following one:

In [12]:
percentages["percentage_all"]

{'Friday': {'0': 0.0025804557028799436,
  '1': 0.0014038967142989678,
  '2': 0.0007284008075281374,
  '3': 0.00045620892682025443,
  '4': 0.00038796927222025003,
  '5': 0.0006314084894730749,
  '6': 0.001543059605421449,
  '7': 0.003437668442686741,
  '8': 0.006557524111983574,
  '9': 0.009010701357969126,
  '10': 0.010298820680755728,
  '11': 0.010461369071488322,
  '12': 0.009388319671345556,
  '13': 0.009566969553612983,
  '14': 0.009629075306675908,
  '15': 0.00874195979687585,
  '16': 0.007916566671292652,
  '17': 0.006809397443849882,
  '18': 0.006461681900466713,
  '19': 0.0064497974662386225,
  '20': 0.0061588205120734355,
  '21': 0.005297390714959896,
  '22': 0.004200955815207016,
  '23': 0.0031685435127473973},
 'Monday': {'0': 0.001897292419187764,
  '1': 0.0010573312774540015,
  '2': 0.0005328826960337426,
  '3': 0.0003362144780011455,
  '4': 0.0002702750364775457,
  '5': 0.0004079044522157569,
  '6': 0.0011198203993629944,
  '7': 0.002971491925868733,
  '8': 0.007006449030

# Creating validation DataFrame 

In [13]:
df_chats_validation = processing.select_time_interval_index(
    df,
    '2021-06-16',
    '2021-07-14',
    'include'
)

## Validation frame visualization
This is the DataFrame with real chats during the specified time period.

It will be used to compare with our predictions for the same time period.

In [14]:
df_chats_validation

,night,weekend,chats,TMA,TME,dropout,ct_agents,week_day,hour
time,,,,,,,,,
2021-06-16 00:00:00,True,False,44,11.597348,0.281439,0.000000,29,Wednesday,0
2021-06-16 01:00:00,True,False,36,13.985784,1.518056,0.055556,27,Wednesday,1
2021-06-16 02:00:00,True,False,13,14.162821,0.330769,0.000000,11,Wednesday,2
2021-06-16 03:00:00,True,False,1,8.433333,0.233333,0.000000,1,Wednesday,3
2021-06-16 04:00:00,True,False,7,16.6,0.433333,0.000000,7,Wednesday,4
...,...,...,...,...,...,...,...,...,...
2021-07-13 20:00:00,False,False,242,14.646963,0.382645,0.024793,61,Tuesday,20
2021-07-13 21:00:00,False,False,232,13.431962,0.418247,0.004310,51,Tuesday,21
2021-07-13 22:00:00,False,False,170,13.611176,0.368333,0.000000,37,Tuesday,22


# Comparing distributions

Each distribution score for the different types of percentages. Therefore, we can asssess which one is the most precise to use on our pipeline.

In [15]:
pd.DataFrame(
    {
        key: evaluations.get_mape_rmse_different_df(frame.chats.values, df_chats_validation.chats.values)
        for key, frame in {
            key:
                feng.set_features_specific_percentage(
                    df=feng.gen_df([datetime(2021, 6, 17), datetime(2021, 7, 15)], ['night','weekend', 'chats', 'TME', 'dropout', 'ct_agents']),
                    chats_per_week=sum(df_chats_validation.chats)/4,
                    percentage_chats=percentage,
                    dropout=0.1,
                    TME=5,
                    WDAY=CONVERT_WEEKDAY
                )
            for key, percentage in percentages.items()
        }.items()
    }
).transpose()

,MAPE,RMSE
percentage_all,33.833019,64.493703
percentage_all_no_hdays,33.937496,64.210556
per_1,33.759793,68.533062
per_1_no_hdays,35.349407,70.834971
per_2,38.000121,69.892581
per_2_no_hdays,38.639498,72.281943
per_3,33.286443,62.355700
per_3_no_hdays,33.016249,60.717818
per_4,34.439411,68.863022
per_4_no_hday,34.439411,68.863022


# Conclusion:
Use period 3 with no holidays (the one after September) to try to predict the proportion of distribution.

# Exporting the best distribution as Json

In [16]:
with open('../configurations/days_chats_distribution.json', 'w') as fp:
    json.dump(percentages["per_3_no_hdays"], fp)

In [17]:
percentages["per_3_no_hdays"]

{'Friday': {'0': 0.0024300189485079965,
  '1': 0.0013406789708342553,
  '2': 0.0006982319834385994,
  '3': 0.0004217590909620337,
  '4': 0.00040398145486043635,
  '5': 0.0007080403343912049,
  '6': 0.0017887980049814162,
  '7': 0.0038362912663378007,
  '8': 0.006982319834385994,
  '9': 0.00937249235714903,
  '10': 0.010684359297060008,
  '11': 0.010653095178398578,
  '12': 0.009748887824955265,
  '13': 0.00984084111513594,
  '14': 0.00974704875915165,
  '15': 0.008905982664965735,
  '16': 0.007916565262621663,
  '17': 0.006773892376643128,
  '18': 0.006679486998724301,
  '19': 0.006475350694523201,
  '20': 0.005820643268436788,
  '21': 0.005100955517289364,
  '22': 0.003980964442888731,
  '23': 0.0030412018172422226},
 'Monday': {'0': 0.0016116346658999806,
  '1': 0.0008943990024907081,
  '2': 0.00043831068319455537,
  '3': 0.000271568717000263,
  '4': 0.00023233531318984124,
  '5': 0.0004278893103074121,
  '6': 0.0011598375001455926,
  '7': 0.002993999128282809,
  '8': 0.0068866884125